In [83]:
from tensorflow.keras.models import model_from_json
import os
import numpy as np
from PIL import Image
from IPython.display import clear_output
import time
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
import pandas as pd
import copy

%run ../utils/object_detection_utils.ipynb
%run ../utils/image_utils.ipynb
%run ../utils/data_utils.ipynb
%run ./variables.ipynb
%run ./utils.ipynb

id_map_path = "./saved_models/model_id_map.csv"
inv_id_map = {v: k for k, v in get_selected_taxons(id_map_path).items()} 
id_map  = get_selected_taxons(id_map_path)

In [2]:
X_test, y_test, _ = get_dataset(TEST_DATASET_PATH, ids=False)

'162/162'

In [3]:
json_file = open("./saved_models/model.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("./saved_models/model.h5")

In [29]:
X_test, y_test, _ = get_dataset(TEST_DATASET_PATH, ids=True)
y_test = np.array(y_test)

'162/162'

In [30]:
thumbails_batch = []
for file in X_test:
    sample = load_image(file, expand=True)
    thumbails_batch.append(sample)
thumbails_batch = np.array(thumbails_batch).astype('float32')/255
outs = loaded_model.predict(thumbails_batch, use_multiprocessing=True)
y_pred = np.array([np.argmax(out) for out in outs])
# printng
#print(X_test[i])
#display(Image.fromarray(sample))
#print("Prediction:",inv_id_map[np.argmax(out)])
#print(np.argmax(out))

In [146]:
conf_mat = tf.math.confusion_matrix(y_test, y_pred).numpy()

In [147]:
conf_mat_perc = conf_mat.copy().astype("float32")
for i in range(len(conf_mat_perc)):
    conf_mat_perc[i] = conf_mat_perc[i]/np.sum(conf_mat_perc[i])

In [148]:
labels = list(id_map.keys())

In [151]:
sn.set()
df_cm = pd.DataFrame(conf_mat_perc, index = [i for i in labels],
                  columns = [i for i in labels])
fig = plt.figure(figsize = (100,70))
sn.heatmap(df_cm,
           annot=True)
           #norm=LogNorm())
           #cbar_kws={'format': FuncFormatter(fmt)})
fig.savefig('foo.png', bbox_inches='tight')
plt.close(fig)